In [1]:
from delta import configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession

In [2]:
builder = (SparkSession.builder
           .appName("merge-delta-table")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-182274dc-734e-4791-82a9-05b5a235384b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 335ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

In [3]:
%load_ext sparksql_magic
%config SparkSql.limit=20

In [4]:
%%sparksql 
CREATE OR REPLACE TABLE default.movie_and_show_titles ( 
    show_id STRING, 
    type STRING, 
    title STRING, 
    director STRING, 
    cast STRING, 
    country STRING, 
    date_added STRING, 
    release_year STRING, 
    rating STRING, 
    duration STRING, 
    listed_in STRING, 
    description STRING  
) USING DELTA LOCATION '/opt/workspace/data/delta_lake/movie_and_show_titles'; 

In [5]:
# For PySpark:
deltaTable_titles = DeltaTable.forPath(spark, "/opt/workspace/data/delta_lake/movie_and_show_titles")

In [6]:
deltaTable_titles.toDF().show(5)

+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



In [8]:
# For PySpark:
df_netflix = spark.read.format("delta").load("/opt/workspace/data/delta_lake/netflix_titles")
df_netflix_deduped = df_netflix.dropDuplicates(["type", "title", "director", "date_added"])

In [9]:
(deltaTable_titles.alias('movie_and_show_titles')
 .merge(df_netflix_deduped.alias('updates')
        ,"""lower(movie_and_show_titles.type) = lower(updates.type)
          AND lower(movie_and_show_titles.title) = lower(updates.title)
          AND lower(movie_and_show_titles.director) = lower(updates.director)
          AND movie_and_show_titles.date_added = updates.date_added""")
 .whenMatchedUpdate(set ={
    "show_id": "updates.show_id",
     "type": "updates.type",
     "title" : "updates.title",
     "director" : "updates.director",
     "cast" : "updates.cast",
     "country" : "updates.country",
     "date_added" : "updates.date_added",
     "release_year" : "updates.release_year",
     "rating" : "updates.rating",
     "duration" : "updates.duration",
     "listed_in" : "updates.listed_in",
     "description" : "updates.description"})
 .whenNotMatchedInsert(values = {
    "show_id": "updates.show_id",
     "type": "updates.type",
     "title" : "updates.title",
     "director" : "updates.director",
     "cast" : "updates.cast",
     "country" : "updates.country",
     "date_added" : "updates.date_added",
     "release_year" : "updates.release_year",
     "rating" : "updates.rating",
     "duration" : "updates.duration",
     "listed_in" : "updates.listed_in",
     "description" : "updates.description"})
  .execute())

In [10]:
%%sparksql
DESCRIBE HISTORY "/opt/workspace/data/delta_lake/movie_and_show_titles"

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2024-02-04 17:08:35.600000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(((lower(type#329) = lower(type#553)) AND (lower(title#330) = lower(title#554))) AND ((lower(director#331) = lower(director#555)) AND (date_added#334 = date_added#558)))""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,0,Serializable,False,"{'numOutputRows': '8806', 'numTargetBytesAdded': '2033266', 'numTargetRowsInserted': '8806', 'numTargetFilesAdded': '2', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesRemoved': '0', 'numTargetRowsMatchedUpdated': '0', 'executionTimeMs': '12669', 'numTargetRowsCopied': '0', 'rewriteTimeMs': '2864', 'numTargetRowsUpdated': '0', 'numTargetRowsDeleted': '0', 'scanTimeMs': '2247', 'numSourceRows': '8806', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '0', 'numTargetBytesRemoved': '0'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
0,2024-02-04 17:06:44.364000,null,null,CREATE OR REPLACE TABLE,"{'description': None, 'partitionBy': '[]', 'properties': '{}', 'isManaged': 'false'}",null,null,null,null,Serializable,True,{},null,Apache-Spark/3.4.1 Delta-Lake/2.4.0


In [11]:
# Read CSV file into a DataFrame
df_titles = (spark.read
      .format("csv")
      .option("header", "true")
      .load("../data/titles.csv"))

In [12]:
df_titles_deduped = df_titles.dropDuplicates(["type", "title"])

In [13]:
df_titles_deduped.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- age_certification: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- seasons: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- imdb_score: string (nullable = true)
 |-- imdb_votes: string (nullable = true)
 |-- tmdb_popularity: string (nullable = true)
 |-- tmdb_score: string (nullable = true)



In [14]:
df_titles_deduped.createOrReplaceTempView("titles_deduped")

In [15]:
(deltaTable_titles.alias('movie_and_show_titles')
 .merge(df_titles_deduped.alias('updates')
        ,"""lower(movie_and_show_titles.type) = lower(updates.type)
          AND lower(movie_and_show_titles.title) = lower(updates.title)
          AND movie_and_show_titles.release_year = updates.release_year""")
 .whenMatchedUpdate(set ={
     "show_id" : "updates.id",
     "type" : "updates.type",
     "title" : "updates.title",
     "country" : "updates.production_countries",
     "release_year" : "updates.release_year",
     "rating" : "updates.age_certification",
     "duration" : "updates.runtime",
     "listed_in" : "updates.genres",
     "description" : "updates.description"})
 .whenNotMatchedInsert(values = {
     "show_id" : "updates.id",
     "type" : "updates.type",
     "title" : "updates.title",
     "country" : "updates.production_countries",
     "release_year" : "updates.release_year",
     "rating" : "updates.age_certification",
     "duration" : "updates.runtime",
     "listed_in" : "updates.genres",
     "description" : "updates.description"})
  .execute())

In [16]:
%%sparksql
MERGE INTO default.movie_and_show_titles
USING titles_deduped
ON lower(default.movie_and_show_titles.type) = lower(titles_deduped.type) 
    AND lower(default.movie_and_show_titles.title) = lower(titles_deduped.title) 
    AND default.movie_and_show_titles.release_year = titles_deduped.release_year
WHEN MATCHED THEN
  UPDATE SET
    show_id = titles_deduped.id,
    type = titles_deduped.type,
    title = titles_deduped.title,
    country = titles_deduped.production_countries,
    release_year = titles_deduped.release_year,
    rating = titles_deduped.age_certification,
    duration = titles_deduped.runtime,
    listed_in = titles_deduped.genres,
    description = titles_deduped.description
WHEN NOT MATCHED
  THEN INSERT (
    show_id,
    type,
    title,
    country,
    release_year,
    rating,
    duration,
    listed_in,
    description
  )
  VALUES (
    titles_deduped.id,
    titles_deduped.type,
    titles_deduped.title,
    titles_deduped.production_countries,
    titles_deduped.release_year,
    titles_deduped.age_certification,
    titles_deduped.runtime,
    titles_deduped.genres,
    titles_deduped.description
  )

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
5899,5826,0,73


In [17]:
%%sparksql
DESCRIBE HISTORY "/opt/workspace/data/delta_lake/movie_and_show_titles"

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2024-02-04 17:08:59.187000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(((lower(type#3518) = lower(type#2328)) AND (lower(title#3519) = lower(title#2327))) AND (release_year#3524 = release_year#2330))""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,2,Serializable,False,"{'numOutputRows': '12557', 'numTargetBytesAdded': '2858145', 'numTargetRowsInserted': '73', 'numTargetFilesAdded': '2', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesRemoved': '2', 'numTargetRowsMatchedUpdated': '5826', 'executionTimeMs': '4850', 'numTargetRowsCopied': '6658', 'rewriteTimeMs': '2142', 'numTargetRowsUpdated': '5826', 'numTargetRowsDeleted': '0', 'scanTimeMs': '2302', 'numSourceRows': '5898', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '0', 'numTargetBytesRemoved': '2850609'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
2,2024-02-04 17:08:50.217000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(((lower(type#329) = lower(type#2328)) AND (lower(title#330) = lower(title#2327))) AND (release_year#335 = release_year#2330))""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,1,Serializable,False,"{'numOutputRows': '12484', 'numTargetBytesAdded': '2850609', 'numTargetRowsInserted': '3678', 'numTargetFilesAdded': '2', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesRemoved': '2', 'numTargetRowsMatchedUpdated': '2221', 'executionTimeMs': '8162', 'numTargetRowsCopied': '6585', 'rewriteTimeMs': '4172', 'numTargetRowsUpdated': '2221', 'numTargetRowsDeleted': '0', 'scanTimeMs': '3259', 'numSourceRows': '5898', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '0', 'numTargetBytesRemoved': '2033266'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
1,2024-02-04 17:08:35.600000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(((lower(type#329) = lower(type#553)) AND (lower(title#330) = lower(title#554))) AND ((lower(director#331) = lower(director#555)) AND (date_added#334 = date_added#558)))""]', 'notMatchedBySourcePredicates': '[]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,0,Serializable,False,"{'numOutputRows': '8806', 'numTargetBytesAdded': '2033266', 'numTargetRowsInserted': '8806', 'numTargetFilesAdded': '2', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesRemoved': '0', 'numTargetRowsMatchedUpdated': '0', 'executionTimeMs': '12669', 'numTargetRowsCopied': '0', 'rewriteTimeMs': '2864', 'numTargetRowsUpdated': '0', 'numTargetRowsDeleted': '0', 'scanTimeMs': '2247', 'numSourceRows': '8806', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '0', 'numTargetBytesRemoved': '0'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
0,2024-02-04 17:06:44.364000,null,null,CREATE OR REPLACE TABLE,"{'description': None, 'partitionBy': '[]', 'properties': '{}', 'isManaged': 'false'}",null,null,null,null,Serializable,True,{},null,Apache-Spark/3.4.1 Delta-Lake/2.4.0


In [18]:
spark.stop()